
# dGen Results — streamlined analysis

This notebook demonstrates a clean, single‑pass workflow built around
`DataWarehouse` and the revised coincident‑peak metrics.

> **Tip:** Edit `ROOT_DIR` and `RUN_ID` to point at your run directory.


In [ ]:
import sys, os
sys.path.append(os.path.abspath("../"))

In [ ]:

from pathlib import Path
import importlib

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import analysis_functions_edit as af
importlib.reload(af)  # safe reload during editing

sns.set_context("talk")
sns.set_style("whitegrid")

ROOT_DIR = "/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs"      
RUN_ID   = "run_ma_instant_permitting_adjust_bass_params_p_007_q_3" 
STATES   = ["MA"]                      

cfg = af.SavingsConfig(lifetime_years=25, cap_to_horizon=False)
states = gpd.read_file("../../../data/states.shp")


## 1) Load once

In [ ]:

wh = af.DataWarehouse.from_disk(ROOT_DIR, run_id=RUN_ID, states=STATES)


## 2) Aggregations and peaks

In [ ]:

outputs = af.aggregate_state_metrics(wh.agents, cfg)
peaks   = af.compute_peaks_by_state(wh.state_hourly)

list(outputs.keys()), peaks.head()


## 3) Coincident peak reduction — *top‑10 baseline hours* (recommended)

In [ ]:

rto_co_top10 = af.compute_rto_coincident_reduction(
    wh, method="baseline_topN_avg", top_n=10, return_by_rto=False
)


## 4) National lines (with coincident delta panel)

In [ ]:

af.facet_lines_national_totals(
    outputs=outputs,
    peaks_df=peaks,
    coincident_df=rto_co_top10,
    metrics=("number_of_adopters","system_kw_cum","batt_kwh_cum","cumulative_bill_savings","peak_mw","coincident_reduction_mw"),
    title="U.S. Totals — BAU vs Policy + Coincident Δ (top-10)",
    ncols=3,
)


## 5) Sensitivity: alternative coincident metrics

In [ ]:

co_single   = af.compute_rto_coincident_reduction(wh, method="single_hour")
co_sep_topN = af.compute_rto_coincident_reduction(wh, method="separate_topN_means", top_n=10)


In [ ]:
af.facet_choropleth_payback_continuous(warehouse=wh, shapefile_path="../../../data/states.shp", year=2040, agents=wh.agents)

In [ ]:
af.plot_us_cum_adopters_grouped(outputs)

In [ ]:
if len(STATES)>1:
    af.choropleth_pct_households_with_solar(outputs, "../../../data/states.shp", year=2040)
else: 
    pass

In [ ]:
if len(STATES)>1:
    af.choropleth_state_coincident_reduction(
        warehouse=wh, shapefile_path="../../../data/states.shp", 
        year=2040, method="baseline_topN_avg", top_n=10)
else: 
    pass

In [ ]:
eabs_states = af.build_eabs_calendar_timeseries(warehouse=wh)
eabs_us = af.build_eabs_calendar_timeseries(warehouse=wh, level="us")

In [ ]:
eabs_states

## 6) Optional: export tidy tables

In [ ]:

out_path = af.export_compiled_results_to_excel(outputs, run_id=RUN_ID or "run", out_dir="/Volumes/Seagate Portabl/permit_power/dgen_runs/compiled_results",
                                              peaks_df=peaks, coincident_df=rto_co_top10)
out_path
